In [14]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky
# from math import e as euler

First get weather data and metadata for a desired location (latitude and logitude)

In [15]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove or comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

c:\ProgramData\anaconda3\lib\site-packages\pvlib\iotools\psm3.py:185: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [17]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 129 # average
sd_Ea = 3.4 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_R0 = 13.7223084 
sd_R0 = 2.47334772

# number of iterations
n = 20000


In [18]:
# notice symmetry of matrix
A = np.array([[1, Ea_X, Ea_lnR0],
              [Ea_X, 1, X_lnR0],
              [Ea_lnR0, X_lnR0, 1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True)

# generating standard distribution of points for each
ea, x, lnR0 = np.random.normal(mean_Ea, sd_Ea, 20_000), np.random.normal(mean_X, sd_X, 20_000), np.random.normal(mean_R0, sd_R0, 20_000)

# transpose built in so I don't have to do it with another function
samples_matrix = np.array([ea, x, lnR0])



In [20]:
print(ea)
print(x)
print(lnR0)

[125.84610798 125.5255213  131.40146668 ... 131.10428891 127.83358942
 124.01265505]
[ 0.05574177 -0.10414759  0.08907726 ... -0.0234405   0.02105521
 -0.08736628]
[12.9409529   7.44493558 12.33626385 ... 14.74105689 15.21587471
 12.70976899]


In [21]:
# correlated stats pre-input to function

# not entirely sure what I should do here,
# do i multiply the transpose by the lower cholesky to get correlated input data for the equation on the spreadsheet
# using MonteCarloEaLnRoX

# this gives us correlated input samples
# looks like what kempe's code was doing in "MonteCarloEaLnRoX" but it is hard to tell what's happening in his code
correlated_samples = np.matmul(A_decomp, samples_matrix)

correlated_df = pd.DataFrame(correlated_samples.T, columns=['ea', 'x', 'lnR0'])

# is back tracking on by default
sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')
# we only care about global irradiance in this case


In [22]:
print(correlated_df)

               ea         x        lnR0
0      125.846108  3.440982 -125.411616
1      125.525521  3.272527 -125.247208
2      131.401467  3.623744 -130.982031
3      135.320921  3.629781 -134.898963
4      131.871743  3.625203 -131.429213
...           ...       ...         ...
19995  129.085663  3.573285 -128.577057
19996  133.674989  3.772920 -133.367388
19997  131.104289  3.503273 -130.614342
19998  127.833589  3.459771 -127.332201
19999  124.012655  3.248606 -123.583854

[20000 rows x 3 columns]


In [23]:
# attempting to compare my function input values to kempe's from macro
# from one trial in his equation calculation loop
# Ea: 69.388
# LnR0: 11.277
# X: 0.0204

# can conclude there is something weird happening with my input data. 

In [3]:
def mikeArrhenius(weather_df, samples): # not using argument properly, it didnt like when i did this

    #['Ea', 'X', 'lnR0']
    # d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** y)).mean() for x, y, z in zip(correlated_df['Ea'], correlated_df['X'], correlated_df['R0'])]
    d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** y)).mean() for x, y, z in zip(samples['ea'], samples['x'], np.log(samples['lnR0']))]

    return pd.DataFrame(d)

In [4]:
temp2 = mikeArrhenius(weather_df=weather_df, samples=correlated_df) # gets mad if i pass as value

# difference is probably something about how I am passing values to the functions,
# the original function takes data as arguments but the second just pulls them from globalscope, this could cause undesired behavior

NameError: name 'mikeArrhenius' is not defined

In [33]:
temp2.head(24) 
# each row shows the degredation rate for that given monte carlo trial iteration
# data frame indexes are integers corresponding to each trial

,0
0,9.962234e-40
1,4.793767e-41
2,3.567561e-39
3,1.286815e-42
4,3.798554e-41
5,9.553057e-42
6,1.576381e-41
7,7.347084e-43
8,3.085377e-43
9,2.899669e-42
